<a href="https://colab.research.google.com/github/dhruvmun/MDM-Recommendation-System/blob/master/Surprise_combinations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install surprise

     |████████████████████████████████| 6.5MB 3.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678592 sha256=58eceb6bfc7ce34e11d6d136dc59b5104bb2d312ca8ec3cc6d88cee2a50fc377
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
from surprise import SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise.model_selection import train_test_split

In [0]:
import pandas as pd

In [3]:
# Load the movielens-1M dataset
data = Dataset.load_builtin('ml-1m')

Dataset ml-1m could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m


In [0]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.20)

In [0]:
def recommendation(algo, trainset, testset):
  # Train the algorithm on the trainset, and predict ratings for the testset
  algo.fit(trainset)
  predictions = algo.test(testset)

  # Then compute RMSE
  rmse = accuracy.rmse(predictions)
  # Then compute MAE
  mae = accuracy.mae(predictions)
  return rmse, mae, predictions

In [0]:
# results = cross_validate(SVD(), data, measures=['RMSE', 'MAE'], cv=5, verbose=False)

##### Calculating predictions for the top methods:

In [12]:
# KNNBaseline

algo = KNNBaseline()
knn_rmse, knn_mae, knn_pred = recommendation(algo, trainset, testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8952
MAE:  0.7062


In [16]:
# SlopeOne

algo = SlopeOne()
slopeone_rmse, slopeone_mae, slopeone_pred = recommendation(algo, trainset, testset)

RMSE: 0.9072
MAE:  0.7149


In [13]:
# SVD

algo = SVD()
svd_rmse, svd_mae, svd_pred = recommendation(algo, trainset, testset)

RMSE: 0.8745
MAE:  0.6866


In [14]:
# SVDpp

algo = SVDpp()
svdpp_rmse, svdpp_mae, svdpp_pred = recommendation(algo, trainset, testset)

RMSE: 0.8621
MAE:  0.6725


In [0]:
pred_df = pd.DataFrame(columns= ['uid', 'iid', 'og_rating', 'svd_rating', 'knn_rating', 'svdpp_rating', 'slopeone_rating'])

In [21]:
num_test = len(svd_pred)
print(num_test)

200042


In [17]:
count = 0
for pred in svd_pred:
  count += 1
  print(pred.uid)
  if count > 5:
    break

4428
4959
6012
4046
1151
3430


In [28]:
svd_pred[0]

Prediction(uid='4428', iid='3264', r_ui=3.0, est=3.464400302513879, details={'was_impossible': False})

In [0]:
for i in range(num_test): 
  svd = svd_pred[i]
  slopeone = slopeone_pred[i]
  knn = knn_pred[i]
  svdpp = svdpp_pred[i]
  df = pd.DataFrame([[svd.uid, svd.iid, svd.r_ui, svd.est, knn.est, svdpp.est, slopeone.est]], columns=['uid', 'iid', 'og_rating', 'svd_rating', 'knn_rating', 'svdpp_rating', 'slopeone_rating'])
  pred_df = pd.concat([df, pred_df], ignore_index=True)

In [37]:
pred_df

,uid,iid,og_rating,svd_rating,knn_rating,svdpp_rating,slopeone_rating
0,2421,480,4.0,3.765975,3.714164,3.915908,3.835640
1,5682,3489,3.0,3.459879,3.559481,3.467551,3.297716
2,3517,1136,5.0,4.553831,4.589899,4.481263,4.593982
3,1141,1584,3.0,3.796377,3.538771,4.285878,3.550342
4,570,2688,2.0,3.244022,3.331249,3.023198,3.004284
...,...,...,...,...,...,...,...
200037,1151,1307,4.0,4.645709,4.359878,4.409605,4.208234
200038,4046,1358,4.0,3.955001,3.941909,3.847858,3.668242
200039,6012,592,1.0,3.104963,3.209230,3.240841,3.368614
200040,4959,1735,5.0,3.589443,3.606219,3.792916,3.371166


In [0]:
# pred_df.to_csv('prediction.csv')

In [0]:
from surprise import NormalPredictor
from surprise.model_selection import GridSearchCV

In [0]:
param_grid = {'n_factors':[50,100,150],'n_epochs':[20,30],  'lr_all':[0.005,0.01],'reg_all':[0.02,0.1]}

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])